In [2]:
import numpy as np
import pandas as pd
import os
import psycopg2
import re
import configparser
import sys
import time
import pickle
from rtree import index
from tqdm import tqdm

class RTreeBuilder:
    def __init__(self, data_dir="../../large_files"):
        """
        Build and save R-trees for spatial datasets
        
        Parameters:
        -----------
        data_dir : str
            Directory for storing R-tree models
        """
        self.data_dir = data_dir
        self.universe_boundaries = {}
        self.dataset_sizes = {}
        
        # Create directories for storing models
        self.rtree_dir = f"{data_dir}/traditional_methods/rtree/models"
        os.makedirs(self.rtree_dir, exist_ok=True)
        
        # Load dataset metadata
        self.load_spatial_statistics()
        
        # R-tree parameters - FIXED the variant specification
        self.rtree_params = {
            'leaf_capacity': 100,
            'fill_factor': 0.8,
            'near_minimum_overlap_factor': 32,
            'variant': 'RT_Star',  # Fixed: no need for rtree.index prefix
            'dimension': 2
        }
    
    def load_spatial_statistics(self):
        """Load dataset information from spatial_statistics.csv"""
        try:
            stats_df = pd.read_csv("../../spatial_statistics.csv")
            for _, row in stats_df.iterrows():
                table_name = row['Table Name']
                total_objects = row['Total Spatial Objects']
                bbox_str = row['Universe Limits (Bounding Box)']
                
                # Parse bounding box
                bbox = self.parse_bbox(bbox_str)
                self.universe_boundaries[table_name] = bbox
                self.dataset_sizes[table_name] = int(total_objects)
                
            print(f"Loaded metadata for {len(self.universe_boundaries)} datasets")
            sys.stdout.flush()
        except Exception as e:
            print(f"Error loading spatial statistics: {e}")
            sys.stdout.flush()
    
    def parse_bbox(self, bbox_str):
        """Parse bounding box string into coordinates"""
        pattern = r"BOX\(([-\d\.]+) ([-\d\.]+),([-\d\.]+) ([-\d\.]+)\)"
        match = re.search(pattern, bbox_str)
        if match:
            xmin = float(match.group(1))
            ymin = float(match.group(2))
            xmax = float(match.group(3))
            ymax = float(match.group(4))
            return (xmin, ymin, xmax, ymax)
        return (-180, -90, 180, 90)  # Default if parsing fails
    
    def connect_to_database(self):
        """Connect to the database containing spatial data"""
        config = configparser.ConfigParser()
        config.read("../../dataset_generation/config.ini")
        db_params = config["database"]
        
        try:
            conn = psycopg2.connect(
                dbname=db_params["dbname"],
                user=db_params["user"],
                password=db_params["password"],
                host=db_params["host"],
                port=db_params["port"]
            )
            return conn
        except psycopg2.Error as e:
            print(f"Database connection error: {e}")
            sys.stdout.flush()
            return None
    
    def build_rtree(self, dataset_name):
        """
        Build an R-tree index for a dataset and save to disk
        
        Parameters:
        -----------
        dataset_name : str
            Name of the dataset to build R-tree for
        
        Returns:
        --------
        dict
            Model metadata including size and parameters
        """
        print(f"Building R-tree for {dataset_name}...")
        sys.stdout.flush()
        
        # Check if dataset exists
        if dataset_name not in self.universe_boundaries:
            raise ValueError(f"Unknown dataset: {dataset_name}")
        
        # Output paths
        rtree_path = f"{self.rtree_dir}/{dataset_name}_rtree"
        level_nodes_path = f"{self.rtree_dir}/{dataset_name}_level_nodes.pkl"
        metadata_path = f"{self.rtree_dir}/{dataset_name}_metadata.json"
        
        # Check if model already exists
        if os.path.exists(f"{rtree_path}.dat") and os.path.exists(level_nodes_path):
            print(f"R-tree model already exists for {dataset_name}")
            if os.path.exists(metadata_path):
                return pd.read_json(metadata_path, typ='series').to_dict()
            return None
        
        # Define the properties for the R-tree
        p = index.Property()
        p.dimension = self.rtree_params['dimension']
        p.leaf_capacity = self.rtree_params['leaf_capacity']
        p.fill_factor = self.rtree_params['fill_factor']
        p.near_minimum_overlap_factor = self.rtree_params['near_minimum_overlap_factor']
        
        # FIXED: Set variant properly using the imported index module
        if self.rtree_params['variant'] == 'RT_Star':
            p.variant = index.RT_Star
        elif self.rtree_params['variant'] == 'RT_Linear':
            p.variant = index.RT_Linear
        elif self.rtree_params['variant'] == 'RT_Quadratic':
            p.variant = index.RT_Quadratic
        else:
            p.variant = index.RT_Star  # Default to RT_Star
            
        p.tight_mbr = True
        
        # Create the R-tree with disk storage
        idx = index.Index(rtree_path, properties=p)
        
        # Store objects for level analysis
        objects = {}
        
        try:
            # Connect to database and load objects
            conn = self.connect_to_database()
            if not conn:
                raise ValueError(f"Failed to connect to database for {dataset_name}")
                
            cursor = conn.cursor()
            
            # Query to get objects with their MBRs
            cursor.execute(f"""
                SELECT id, ST_XMin(geometry), ST_YMin(geometry), 
                    ST_XMax(geometry), ST_YMax(geometry)
                FROM {dataset_name}_mbr
            """)
            
            # Process in batches
            batch_size = 10000
            total_objects = self.dataset_sizes[dataset_name]
            total_processed = 0
            valid_objects = 0
            point_objects = 0
            invalid_objects = 0
            
            print(f"Loading {total_objects} objects for {dataset_name}...")
            sys.stdout.flush()
            
            # Use a progress counter
            progress_step = max(1, total_objects // 10)
            
            # Small buffer to add to point geometries
            POINT_BUFFER = 0.000001  # Small epsilon for point geometries
            
            start_time = time.time()
            
            while True:
                batch_data = cursor.fetchmany(batch_size)
                if not batch_data:
                    break
                    
                for obj in batch_data:
                    obj_id, xmin, ymin, xmax, ymax = obj
                    
                    # Skip null geometries
                    if xmin is None or ymin is None or xmax is None or ymax is None:
                        invalid_objects += 1
                        continue
                    
                    # Handle point geometries (where min = max)
                    is_point = False
                    if xmin == xmax and ymin == ymax:
                        is_point = True
                        point_objects += 1
                        # Add a small buffer to make it a valid rectangle for R-tree
                        xmin -= POINT_BUFFER
                        ymin -= POINT_BUFFER
                        xmax += POINT_BUFFER
                        ymax += POINT_BUFFER
                    
                    # Handle invalid geometries where min > max
                    if xmin > xmax:
                        mid = (xmin + xmax) / 2
                        xmin, xmax = mid - POINT_BUFFER, mid + POINT_BUFFER
                    
                    if ymin > ymax:
                        mid = (ymin + ymax) / 2
                        ymin, ymax = mid - POINT_BUFFER, mid + POINT_BUFFER
                    
                    # Final validation
                    if xmin >= xmax or ymin >= ymax:
                        invalid_objects += 1
                        continue
                        
                    # Insert into R-tree
                    try:
                        idx.insert(obj_id, (xmin, ymin, xmax, ymax))
                        
                        # Store in objects dictionary (for level analysis)
                        objects[obj_id] = (xmin, ymin, xmax, ymax)
                        valid_objects += 1
                    except Exception as e:
                        invalid_objects += 1
                        continue
                
                total_processed += len(batch_data)
                
                # Simple progress reporting
                if total_processed % progress_step == 0:
                    print(f"  {min(total_processed, total_objects) / total_objects * 100:.1f}% loaded... "
                        f"(valid: {valid_objects}, points: {point_objects}, invalid: {invalid_objects})", 
                        end="\r", flush=True)
                    sys.stdout.flush()
            
            build_time = time.time() - start_time
            print(f"\nR-tree building complete in {build_time:.2f} seconds. "
                  f"Valid objects: {valid_objects} (including {point_objects} points), "
                  f"Invalid geometries: {invalid_objects}")
            sys.stdout.flush()
            conn.close()
            
            # Check if we have enough valid objects
            if valid_objects == 0:
                print(f"No valid objects found for {dataset_name}. Using fallback mechanism.")
                level_nodes = self.create_fallback_nodes(dataset_name)
            else:
                # Generate level statistics for estimation
                level_nodes = self.generate_level_nodes(dataset_name, idx, objects, self.rtree_params['leaf_capacity'])
            
            # Save level nodes to disk
            with open(level_nodes_path, 'wb') as f:
                pickle.dump(level_nodes, f)
            
            # Calculate model size
            model_size_disk = 0
            if os.path.exists(f"{rtree_path}.dat"):
                model_size_disk += os.path.getsize(f"{rtree_path}.dat")
            if os.path.exists(f"{rtree_path}.idx"):
                model_size_disk += os.path.getsize(f"{rtree_path}.idx")
                
            level_nodes_size = os.path.getsize(level_nodes_path)
            
            # Calculate memory usage estimate
            memory_per_entry = 44  # Bytes per entry (2 coordinates * 2 * 8 bytes + indexing overhead)
            memory_usage = valid_objects * memory_per_entry
            
            # Compute number of nodes at each level (estimated)
            est_leaf_nodes = np.ceil(valid_objects / self.rtree_params['leaf_capacity'])
            leaf_level = int(np.ceil(np.log(max(1, est_leaf_nodes)) / 
                                    np.log(max(2, self.rtree_params['leaf_capacity']))))
            
            # Create detailed model metadata
            metadata = {
                'dataset': dataset_name,
                'total_objects': total_objects,
                'valid_objects': valid_objects,
                'point_objects': point_objects,
                'invalid_objects': invalid_objects,
                'model_size_bytes': model_size_disk,
                'level_nodes_size_bytes': level_nodes_size,
                'total_size_bytes': model_size_disk + level_nodes_size,
                'estimated_memory_bytes': memory_usage,
                'estimated_levels': leaf_level + 1,
                'build_time_seconds': build_time,
                'rtree_params': self.rtree_params,
                'num_level_nodes': len(level_nodes),
                'grid_size': int(np.sqrt(len(level_nodes))) if len(level_nodes) > 0 else 0
            }
            
            # Save metadata
            pd.Series(metadata).to_json(metadata_path)
            
            print(f"R-tree built successfully for {dataset_name}")
            print(f"Model size: {metadata['total_size_bytes'] / (1024*1024):.2f} MB")
            print(f"Estimated memory usage: {metadata['estimated_memory_bytes'] / (1024*1024):.2f} MB")
            print(f"Number of level nodes: {metadata['num_level_nodes']}")
            sys.stdout.flush()
            
            return metadata
            
        except Exception as e:
            print(f"Error building R-tree for {dataset_name}: {e}")
            sys.stdout.flush()
            
            # Create fallback
            level_nodes = self.create_fallback_nodes(dataset_name)
            with open(level_nodes_path, 'wb') as f:
                pickle.dump(level_nodes, f)
            
            return None
    
    def create_fallback_nodes(self, dataset_name):
        """Create simple grid nodes when R-tree building fails"""
        print(f"Creating fallback estimation nodes for {dataset_name}")
        sys.stdout.flush()
        
        if dataset_name not in self.universe_boundaries:
            return []
        
        universe = self.universe_boundaries[dataset_name]
        total_objects = self.dataset_sizes[dataset_name]
        
        # Create a simple 4x4 grid of nodes covering the universe
        xmin, ymin, xmax, ymax = universe
        width = xmax - xmin
        height = ymax - ymin
        
        if width <= 0 or height <= 0:
            # Use default values if universe is invalid
            xmin, ymin = -180, -90
            xmax, ymax = 180, 90
            width = 360
            height = 180
        
        grid_size = 4
        cell_width = width / grid_size
        cell_height = height / grid_size
        nodes = []
        
        # Create a uniform grid
        for i in range(grid_size):
            for j in range(grid_size):
                cell_xmin = xmin + i * cell_width
                cell_ymin = ymin + j * cell_height
                cell_xmax = cell_xmin + cell_width
                cell_ymax = cell_ymin + cell_height
                
                # Assume uniform distribution
                estimated_objects = total_objects / (grid_size * grid_size)
                
                nodes.append({
                    'mbr': (cell_xmin, cell_ymin, cell_xmax, cell_ymax),
                    'objects': estimated_objects,
                    'area': cell_width * cell_height
                })
        
        print(f"Created {len(nodes)} fallback nodes for {dataset_name}")
        sys.stdout.flush()
        return nodes
    
    def generate_level_nodes(self, dataset_name, idx, objects, leaf_capacity):
        """
        Generate nodes at the level before leaves using grid-based spatial clustering
        """
        print(f"Generating level nodes for {dataset_name}...")
        sys.stdout.flush()
        
        universe = self.universe_boundaries[dataset_name]
        total_objects = len(objects)
        
        # If we have very few objects, just use the universe as one node
        if total_objects <= leaf_capacity * 2:
            return [{
                'mbr': universe,
                'objects': total_objects,
                'area': (universe[2] - universe[0]) * (universe[3] - universe[1])
            }]
        
        # Calculate grid size based on heuristic
        est_leaf_nodes = np.ceil(total_objects / leaf_capacity)
        est_parent_nodes = np.ceil(est_leaf_nodes / leaf_capacity)
        grid_size = max(2, int(np.sqrt(est_parent_nodes)))
        
        print(f"Using {grid_size}x{grid_size} grid to generate level nodes")
        sys.stdout.flush()
        
        univ_xmin, univ_ymin, univ_xmax, univ_ymax = universe
        grid_width = (univ_xmax - univ_xmin) / grid_size
        grid_height = (univ_ymax - univ_ymin) / grid_size
        
        # First pass: find density of each grid cell
        grid_counts = np.zeros((grid_size, grid_size))
        object_count = 0
        
        # Sample a subset of objects for better performance
        sample_size = min(50000, total_objects)
        sample_ids = list(objects.keys())
        if total_objects > sample_size:
            sample_ids = np.random.choice(sample_ids, sample_size, replace=False)
        
        for obj_id in sample_ids:
            obj = objects[obj_id]
            xmin, ymin, xmax, ymax = obj
            
            # Use center point for grid assignment
            x_center = (xmin + xmax) / 2
            y_center = (ymin + ymax) / 2
            
            # Skip objects outside universe
            if (x_center < univ_xmin or x_center >= univ_xmax or
                y_center < univ_ymin or y_center >= univ_ymax):
                continue
                
            # Calculate grid indices
            grid_x = min(grid_size - 1, int((x_center - univ_xmin) / grid_width))
            grid_y = min(grid_size - 1, int((y_center - univ_ymin) / grid_height))
            
            # Increment count
            grid_counts[grid_x, grid_y] += 1
            object_count += 1
        
        # Convert counts to density factors
        if object_count > 0:
            grid_counts = grid_counts / object_count * total_objects
        
        # Second pass: create nodes based on grid cells
        nodes = []
        
        for i in range(grid_size):
            for j in range(grid_size):
                # Calculate node MBR
                node_xmin = univ_xmin + i * grid_width
                node_ymin = univ_ymin + j * grid_height
                node_xmax = node_xmin + grid_width
                node_ymax = node_ymin + grid_height
                
                # Calculate node statistics
                node_mbr = (node_xmin, node_ymin, node_xmax, node_ymax)
                node_area = grid_width * grid_height
                estimated_objects = int(grid_counts[i, j])
                
                # Create node if it has objects
                if estimated_objects > 0:
                    nodes.append({
                        'mbr': node_mbr,
                        'objects': estimated_objects,
                        'area': node_area
                    })
        
        print(f"Generated {len(nodes)} level nodes for {dataset_name}")
        sys.stdout.flush()
        return nodes
    
    def build_all_rtrees(self):
        """Build and save R-trees for all datasets"""
        dataset_names = list(self.universe_boundaries.keys())
        
        print(f"Building R-trees for {len(dataset_names)} datasets")
        sys.stdout.flush()
        
        all_metadata = []
        
        for idx, dataset_name in enumerate(dataset_names, start=1):
            print("\n" + "="*80)
            print(f"DATASET {idx}/{len(dataset_names)}: {dataset_name}")
            print("="*80)
            sys.stdout.flush()
            
            try:
                metadata = self.build_rtree(dataset_name)
                if metadata:
                    all_metadata.append(metadata)
                print(f"Finished processing {dataset_name} ({idx}/{len(dataset_names)})")
                sys.stdout.flush()
            except Exception as e:
                print(f"Error building R-tree for {dataset_name}: {e}")
                print("Moving to next dataset")
                sys.stdout.flush()
        
        # Save combined metadata
        if all_metadata:
            pd.DataFrame(all_metadata).to_csv(f"{self.rtree_dir}/all_rtree_metadata.csv", index=False)
            print("\nSaved metadata for all R-trees")
        else:
            print("No R-trees were built successfully")
        
        return all_metadata

if __name__ == "__main__":
    builder = RTreeBuilder()
    builder.build_all_rtrees()
    print("All R-trees built and saved!")

Loaded metadata for 14 datasets
Building R-trees for 14 datasets

DATASET 1/14: yago2
Building R-tree for yago2...
R-tree model already exists for yago2
Finished processing yago2 (1/14)

DATASET 2/14: craftwaysorted
Building R-tree for craftwaysorted...
R-tree model already exists for craftwaysorted
Finished processing craftwaysorted (2/14)

DATASET 3/14: zcta5
Building R-tree for zcta5...
R-tree model already exists for zcta5
Finished processing zcta5 (3/14)

DATASET 4/14: areawater
Building R-tree for areawater...
R-tree model already exists for areawater
Finished processing areawater (4/14)

DATASET 5/14: aerowaythingnodesorted
Building R-tree for aerowaythingnodesorted...
R-tree model already exists for aerowaythingnodesorted
Finished processing aerowaythingnodesorted (5/14)

DATASET 6/14: emergencythingwaysorted
Building R-tree for emergencythingwaysorted...
R-tree model already exists for emergencythingwaysorted
Finished processing emergencythingwaysorted (6/14)

DATASET 7/14: hi